## Task 2: Develop Time Series Forecasting Models

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Add the parent directory to the system path 
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))

In [ ]:
# Load the preprocessed data
df_TSLA = pd.read_csv('../data/processed/TSLA_processed.csv')
df_BND = pd.read_csv('../data/processed/BND_processed.csv')
df_SPY = pd.read_csv('../data/processed/SPY_processed.csv')

In [ ]:
df_TSLA.head()

In [ ]:
df_BND.head()

In [ ]:
df_SPY.head()

In [ ]:
# Split the data into training and testing sets
train_size = int(len(df_SPY) * 0.8)
df_SPY_train = df_SPY.iloc[:train_size]
df_SPY_test = df_SPY.iloc[train_size:]

### ARIMA MODEL 

In [ ]:
# ARIMA Model
print("Developing ARIMA Model for SPY")
arima_model = ARIMA(df_SPY_train['Close'], order=(1, 1, 1))
arima_model_fit = arima_model.fit()
#arima_forecast = arima_model_fit.forecast(steps=len(df_SPY_test))[0]
arima_forecast = arima_model_fit.forecast(steps=len(df_SPY_test))


In [ ]:
arima_mae = mean_absolute_error(df_SPY_test['Close'], arima_forecast)
arima_rmse = np.sqrt(mean_squared_error(df_SPY_test['Close'], arima_forecast))
arima_mape = mean_absolute_percentage_error(df_SPY_test['Close'], arima_forecast)

In [ ]:
print(f"ARIMA Model Performance:")
print(f"MAE: {arima_mae:.2f}")
print(f"RMSE: {arima_rmse:.2f}")
print(f"MAPE: {arima_mape:.2f}%")

### SARIMA Model

In [ ]:
# SARIMA Model
print("Developing SARIMA Model for SPY")
sarima_model = SARIMAX(df_SPY_train['Close'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sarima_model_fit = sarima_model.fit()
#sarima_forecast = sarima_model_fit.forecast(steps=len(df_SPY_test))[0]
sarima_forecast = sarima_model_fit.forecast(steps=len(df_SPY_test))

sarima_mae = mean_absolute_error(df_SPY_test['Close'], sarima_forecast)
sarima_rmse = np.sqrt(mean_squared_error(df_SPY_test['Close'], sarima_forecast))
sarima_mape = mean_absolute_percentage_error(df_SPY_test['Close'], sarima_forecast)

print(f"SARIMA Model Performance:")
print(f"MAE: {sarima_mae:.2f}")
print(f"RMSE: {sarima_rmse:.2f}")
print(f"MAPE: {sarima_mape:.2f}%")

In [ ]:
# confidence intervals to assess forecast reliability.
sarima_model_fit.get_forecast(steps=len(df_SPY_test)).conf_int()

Plot  for the actual values of df_SPY_test['Close'] against the sarima_forecast to visually assess the accuracy of the SARIMA model.

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_SPY_train['Close'], label='Train Data')
plt.plot(df_SPY_test['Close'], label='Test Data', color='orange')
plt.plot(df_SPY_test.index, sarima_forecast, label='Forecast', color='green')
plt.title("SPY Close Price - Actual vs. SARIMA Forecast")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.legend()
plt.show()


Residuals (actual minus predicted values) to check for patterns

In [ ]:
residuals = df_SPY_test['Close'] - sarima_forecast
plt.figure(figsize=(12, 6))
plt.plot(residuals)
plt.axhline(y=0, color='red', linestyle='--')
plt.title("SARIMA Model Residuals")
plt.xlabel("Date")
plt.ylabel("Residuals")
plt.show()

Residual Autocorrelation (ACF) Plot

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(residuals)
plt.title("ACF of SARIMA Residuals")
plt.show()

Displaying confidence intervals around forecasted values to provide insights into the forecast’s uncertainty.

In [ ]:
sarima_forecast_ci = sarima_model_fit.get_forecast(steps=len(df_SPY_test)).conf_int()
plt.figure(figsize=(12, 6))
plt.plot(df_SPY_test['Close'], label="Actual Test Data", color="orange")
plt.plot(df_SPY_test.index, sarima_forecast, label="Forecasted Values", color="green")
plt.fill_between(df_SPY_test.index, 
                 sarima_forecast_ci.iloc[:, 0], 
                 sarima_forecast_ci.iloc[:, 1], color='k', alpha=0.1)
plt.title("SPY Close Price Forecast with Confidence Intervals")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.legend()
plt.show()


seasonal decomposition

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(df_SPY_train['Close'], model='additive', period=12)
decomposition.plot()
plt.show()

### LSTM Model

In [ ]:
# LSTM Model
print("Developing LSTM Model for SPY")
lstm_model = Sequential()
lstm_model.add(LSTM(50, input_shape=(1, 1), return_sequences=False))  # Change input_shape to (1, 1)
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')

# Reshape X_train to (len(df_SPY_train), 1, 1)
X_train = df_SPY_train[['Close']].values.reshape((len(df_SPY_train), 1, 1))
y_train = df_SPY_train['Close'].values

# Train the model
lstm_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Reshape X_test to (len(df_SPY_test), 1, 1)
X_test = df_SPY_test[['Close']].values.reshape((len(df_SPY_test), 1, 1))
lstm_forecast = lstm_model.predict(X_test)

# Evaluate the model
lstm_mae = mean_absolute_error(df_SPY_test['Close'], lstm_forecast.flatten())
lstm_rmse = np.sqrt(mean_squared_error(df_SPY_test['Close'], lstm_forecast.flatten()))
lstm_mape = mean_absolute_percentage_error(df_SPY_test['Close'], lstm_forecast.flatten())

print(f"LSTM Model Performance:")
print(f"MAE: {lstm_mae:.2f}")
print(f"RMSE: {lstm_rmse:.2f}")
print(f"MAPE: {lstm_mape:.2f}%")


LSTM Actual vs. Forecasted Plot

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_SPY_test.index, df_SPY_test['Close'], label="Actual Test Data", color="orange")
plt.plot(df_SPY_test.index, lstm_forecast, label="LSTM Forecast", color="green")
plt.title("SPY Close Price - Actual vs. LSTM Forecast")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.legend()
plt.show()

LSTM Loss Curve 

In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
plt.plot(history.history['loss'])
plt.title('LSTM Model Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

LSTM Residual Plot 

In [ ]:
lstm_residuals = df_SPY_test['Close'] - lstm_forecast.flatten()
plt.figure(figsize=(12, 6))
plt.plot(lstm_residuals)
plt.axhline(y=0, color='red', linestyle='--')
plt.title("LSTM Model Residuals")
plt.xlabel("Date")
plt.ylabel("Residuals")
plt.show()

In [ ]:
plt.hist(lstm_residuals, bins=30)
plt.title("Distribution of LSTM Residuals")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.show()

In [ ]:
plt.scatter(df_SPY_test['Close'], lstm_forecast.flatten(), alpha=0.5)
plt.plot([df_SPY_test['Close'].min(), df_SPY_test['Close'].max()], 
         [df_SPY_test['Close'].min(), df_SPY_test['Close'].max()], 'r--')
plt.xlabel("Actual Close Price")
plt.ylabel("Predicted Close Price")
plt.title("Actual vs. Predicted Close Price")
plt.show()

### Model Performance Comparison 

In [ ]:
# Compare Model Performance
print("Comparing Model Performance:")
print(f"ARIMA MAE: {arima_mae:.2f}, RMSE: {arima_rmse:.2f}, MAPE: {arima_mape:.2f}%")
print(f"SARIMA MAE: {sarima_mae:.2f}, RMSE: {sarima_rmse:.2f}, MAPE: {sarima_mape:.2f}%")
print(f"LSTM MAE: {lstm_mae:.2f}, RMSE: {lstm_rmse:.2f}, MAPE: {lstm_mape:.2f}%")

In [ ]:
# Select the best model based on performance metrics
if arima_mape < sarima_mape and arima_mape < lstm_mape:
    best_model = "ARIMA"
    best_forecast = arima_forecast
elif sarima_mape < arima_mape and sarima_mape < lstm_mape:
    best_model = "SARIMA"
    best_forecast = sarima_forecast
else:
    best_model = "LSTM"
    best_forecast = lstm_forecast.flatten()

print(f"The best model is {best_model} based on MAPE.")

In [ ]:
print(f"The best model is {best_model} based on MAPE.")

# Visualize the forecasts
plt.figure(figsize=(12, 6))
plt.plot(df_SPY_test.index, df_SPY_test['Close'], label='Actual')
plt.plot(df_SPY_test.index, best_forecast, label='Forecast')
plt.title(f"SPY Stock Price Forecast ({best_model})")
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()